# Seafloor Spreading Bathymetry and Magnetic Anomalies

**Our goals for today:**
- Gain more experience loading, examining, and clearning data using ```pandas```.
- Load marine geophysical data (bathymetry and marine magnetic anomalies) from two oceanic ridges. These are data types that underlie our understanding of seafloor spreading.
- Plot bathymetry data and inspect to evaluate spreading rate.
- Declare polynomial functions to detrend magnetic anomaly data.
- Gain experience writing functions.
- Plot marine magnetic anomaly data and calculate spreading rates.

## Setup

Run this cell as it is to setup your environment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs

## Marine Geology, Bathymetry and Magnetic Anomalies

We'll look at marine magnetics and bathymetry data from two surveys, from the Mid-Atlantic Ridge and East Pacific Rise.

The seafloor age model that you have been working with has been constructed from utilizing the record of Earth's flipping magnetic field that is preserved in the frozen lava/magma that constructed the oceanic crust.

<img src="Figures/diagram-magnetic-reversals-ocean-ridge.jpg" width=900>
> Source: Enduring Resources for Earth Science Education, CC BY 4.0

This video gives a very nice overview of the concepts of continental drift, sea floor spreading and the magnetic striping anomalies that we will examine in this notebook: https://youtu.be/JJEZ3Vizdww It is assigned as part of the out-of-class work this next week.

First we'll load the Atlantic data (that can be accessed here: https://www.ncei.noaa.gov/maps/geophysics), and then we'll need to clean them up.

The data file has the following columns:

```
['SURVEY_ID','TIMEZONE','DATE','TIME','LAT','LON','POS_TYPE','NAV_QUALCO','BAT_TTIME',
'CORR_DEPTH','BAT_CPCO','BAT_TYPCO','BAT_QUALCO','MAG_TOT','MAG_TOT2','MAG_RES',
'MAG_RESSEN','MAG_DICORR','MAG_SDEPTH','MAG_QUALCO','GRA_OBS','EOTVOS','FREEAIR',
'GRA_QUALCO','LINEID','POINTID']
```

In [ ]:
vanc05mv_data_file = pd.read_table('./data_tracks/vanc05mv.m77t')

To make this DataFrame a bit more compact, let's drop the columns that we will not be using.

In [ ]:
atlantic_data = vanc05mv_data_file.drop(columns=['SURVEY_ID','TIMEZONE','DATE','TIME','POS_TYPE','NAV_QUALCO',
                                                 'BAT_TTIME','BAT_CPCO','BAT_TYPCO','BAT_QUALCO','MAG_TOT2','MAG_RES',
                                                 'MAG_RESSEN','MAG_DICORR','MAG_SDEPTH','MAG_QUALCO','GRA_OBS','EOTVOS',
                                                 'FREEAIR','GRA_QUALCO','LINEID','POINTID'])

Use the `.head()` function to inspect the `atlantic_data` DataFrame.

In [ ]:
#Inspect the dataframe using .head()


Use the `.describe()` function applied to the DataFrame to examine the data

In [ ]:
#Use .describe() to examine the data


Notice that some entries do not have either depth, magnetization  or both and instead in those fields `'NaN'` (not a number) is specified. The function ```np.isnan()``` identifies NaN entries.  Applying that function will give us a list of `True` and `False` (a boolean list). 

In [ ]:
np.isnan(atlantic_data['CORR_DEPTH'])

We don't want the ```'NaN'``` values so we can used the complement (`~`) in the logical expression ```~np.isnan``` to find rows where there it is **not** ```'NaN'```.

In [ ]:
~np.isnan(atlantic_data['CORR_DEPTH'])

We can do the same filtering using ```np.isfinite()``` which looks for values that are not infinity or not ```'NaN'```.

In [ ]:
np.isfinite(atlantic_data['CORR_DEPTH'])

We want to have depth AND magnetic field measurements leaving only rows that have BOTH CORR_DEPTH and MAG_TOT specified. That is accomplished in the code cell below where we ask pandas to filter the DataFrame for values where `np.isfinite() = True` for both ```'CORR_DEPTH'``` and ```'MAG_TOT'```.

In [ ]:
atlantic_data_clean = atlantic_data[np.isfinite(atlantic_data['CORR_DEPTH']) &  np.isfinite(atlantic_data['MAG_TOT'])];

Use the `.describe()` function applied to the DataFrame to examine the "cleaned" data. How many points are left now that we are requiring that there is both depth and magnetic data?

Let's take a look at our data!

<font color=goldenrod>**_Code for you to write_**</font>

**Plot atlantic_data on a map and make it have a linewidth of 2 and a color of orange**

**Plot atlantic_data_clean on the same map and make it have a linewidth of 4 and a color of red**

**Add a label to each plotted line ```label='description of data'``` so that they are keyed out in the legend.***

In [ ]:
plt.figure(1,(15,15))
ax = plt.axes(projection=ccrs.Robinson())
ax.set_global()

#ax.plot(ADD_CODE_HERE, transform=ccrs.PlateCarree())
#ax.plot(ADD_CODE_HERE,transform=ccrs.PlateCarree())
ax.coastlines()
ax.stock_img()
ax.gridlines()
plt.legend()

plt.show()

Next we want to plot the cleaned data in profile with an x-axis of longitude.

In [ ]:
plt.figure(1,(15,10))
ax1=plt.subplot(2, 1, 1)
#ax1.plot(..., ..., color='mediumblue')
ax1.set_ylabel('Bathymetry, m')
ax1.set_title('Mid-Atlantic Ridge')

ax2=plt.subplot(2, 1, 2)
#ax2.plot(..., ..., color='darkred')
ax2.set_xlabel('Longitude, degrees')
ax2.set_ylabel('Total magnetic field, nT')

Let's just analyze the portion of the survey from around the ridge, so from longitudes -24.0 to 0.0 degrees. Use Boolean indexing to pull out rows of `atlantic_data_clean` where `atlantic_data_clean['LON']` is between those values.

<font color=goldenrod>**_Code for you to write_**</font>

In [ ]:
#atlantic_data_cropped = atlantic_data_clean[...]

Here's a map of where our survey line was collected with a grid of seafloor bathymetry in the background.

<img src="Figures/MAR_track_map.png" width=900>

In [ ]:
plt.figure(1,(15,10))
ax1=plt.subplot(2, 1, 1)
ax1.plot(atlantic_data_cropped['LON'],-1*atlantic_data_cropped['CORR_DEPTH'],color='mediumblue');
ax1.set_ylabel('Bathymetry, m');
ax1.set_title('Mid-Atlantic Ridge')

ax2=plt.subplot(2, 1, 2)
ax2.plot(atlantic_data_cropped['LON'],atlantic_data_cropped['MAG_TOT'],color='darkred');
ax2.set_xlabel('Longitude, degrees');
ax2.set_ylabel('Total magnetic field, nT');

We need to convert the position of the ```LAT``` , ```LON``` magnetic track into a distance magnetic track referenced to the position of the spreading center. The position of the spreading center is assigned a distance of zero.  Since the data is already presented as a track we can simply find the distance of ```LAT``` , ```LON``` points along the track relative to the ridge position. To start, we create numpy arrays of lat, lon, bathymetry and magnetic intensity.

In [ ]:
#Convert data to numpy arrays
atlantic_lat = np.array(atlantic_data_cropped['LAT'])
atlantic_lon = np.array(atlantic_data_cropped['LON'])
atlantic_bath = np.array(atlantic_data_cropped['CORR_DEPTH'])
atlantic_mag = np.array(atlantic_data_cropped['MAG_TOT'])

We then use the haversine function defined in the next cell to compute the distance between two points on a sphere.

In [ ]:
#We need a function to calculate distance between two points on a sphere. 
#Functions (or subroutines) are a good way to compartmentalize code for repeated operations
#DO NOT EDIT THIS CELL BUT DO READ THROUGH IT TO UNDERSTAND CONSTRUCTION
def haversine_distance(lon1, lat1, lon2, lat2):
    """
    A FUNCTION SHOULD HAVE A COMMENT BLOCK THAT STATES INPUT PARAMETERS AND 
    UNITS AND OUTPUT PARAMETERS AND UNITS
    AS WELL AS DESCRIBE WHAT IT DOES
    
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees).

    All args must be of equal length. i.e. floating point number, or arrays of numbers of equal length
    
    Parameters
    ------------
    lon1 : longitude of point 1 (in decimal degrees)
    lat1 : latitude of point 1 (in decimal degrees)
    lon2 : longitude of point 2 (in decimal degrees)
    lat2 : latitude of point 2 (in decimal degrees)

    Returns
    ------------
    distance : distance between point 1 and 2 in km
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    
    signfac=-dlon/np.abs(dlon)  #note the minus is because the difference of smaller number is positive
    km = 6371 * c * signfac
    return km

Apply this ```haversine_distance()``` function to the distance between Berkeley (237.727, 37.8715) and your birthplace.

Let's now use the ```haversine_distance()``` function to calculate distance from the ridge for all points in the profile.

In [ ]:
#Specify the position of the ridge
ridgelon = -10.07
ridgelat = -48.05926

#create lat2, lon2 arrays of the ridge position. 
#We do this to take advantage of the speed of vector arithmatic in python.
lat2 = np.ones(len(atlantic_lat))*ridgelat
lon2 = np.ones(len(atlantic_lat))*ridgelon
atlantic_dist = haversine_distance(atlantic_lon,atlantic_lat,lon2,lat2)

In [ ]:
#Plot the bathymetry and magnetic field data
plt.figure(1,(15,10))
ax1=plt.subplot(2, 1, 1)
ax1.plot(atlantic_dist,-1*atlantic_bath,color='mediumblue')
ax1.grid()
ax1.set_title('Mid Atlantic Ridge')
ax1.set_ylabel('Bathymentry, m')

ax2=plt.subplot(2, 1, 2)
ax2.plot(atlantic_dist,atlantic_mag,color='darkred')
ax2.set_xlabel('Distance to Ridge, km')
ax2.set_ylabel('Total magnetic field, nT')
plt.show()

### Next lets load a data set for the East Pacific rise to compare with

In [ ]:
# Load the seafloor depth, marine mag anom data
# Source: https://maps.ngdc.noaa.gov/viewers/geophysics/
#names=['SURVEY_ID','TIMEZONE','DATE','TIME','LAT','LON','POS_TYPE','NAV_QUALCO','BAT_TTIME','CORR_DEPTH','BAT_CPCO','BAT_TYPCO','BAT_QUALCO','MAG_TOT','MAG_TOT2','MAG_RES','MAG_RESSEN','MAG_DICORR','MAG_SDEPTH','MAG_QUALCO','GRA_OBS','EOTVOS','FREEAIR','GRA_QUALCO','LINEID','POINTID'])

nbp9707_data_file=pd.read_table('data_tracks/nbp9707.m77t')
pacific_data = nbp9707_data_file.drop(columns=['SURVEY_ID','TIMEZONE','DATE','TIME','POS_TYPE','NAV_QUALCO','BAT_TTIME','BAT_CPCO','BAT_TYPCO','BAT_QUALCO','MAG_TOT2','MAG_RES','MAG_RESSEN','MAG_DICORR','MAG_SDEPTH','MAG_QUALCO','GRA_OBS','EOTVOS','FREEAIR','GRA_QUALCO','LINEID','POINTID'])

pacific_data_clean = pacific_data[~np.isnan(pacific_data['CORR_DEPTH']) &  ~np.isnan(pacific_data['MAG_TOT'])]; #use ~np.isnan to clear out rows were there are nans
pacific_data_cropped = pacific_data_clean[(pacific_data_clean['LON']>-126.0) & (pacific_data_clean['LON']<-95.0)] # use Boolean indexing to select rows with Longitude -126 deg to -95 deg

Here's a map of where our survey line was collected with a grid of seafloor bathymetry in the background.

<img src="Figures/EPR_track_map.png" width=900>

In [ ]:
plt.figure(1,(15,10))
ax1=plt.subplot(2, 1, 1)
#ax1.plot(pacific_data_cropped['LON'],-1*pacific_data_cropped['CORR_DEPTH'],color='blue')
ax1.set_ylabel('Bathymetry, m')
ax1.set_title('East Pacific Rise')

ax2=plt.subplot(2, 1, 2)
#ax2.plot(pacific_data_cropped['LON'],pacific_data_cropped['MAG_TOT'],color='blue')
ax2.set_xlabel('Longitude, degrees')
ax2.set_ylabel('Total magnetic field, nT')
plt.show()

In [ ]:
#Specify the position of the ridge
ridgelon = -113.521
tmp = np.array(pacific_data_cropped['LAT'][np.abs(pacific_data_cropped['LON'] - ridgelon) < 0.01])
ridgelat = tmp[0]

#Convert data to numpy arrays
pacific_lat = np.array(pacific_data_cropped['LAT'])
pacific_lon = np.array(pacific_data_cropped['LON'])
pacific_bath = np.array(pacific_data_cropped['CORR_DEPTH'])
pacific_mag = np.array(pacific_data_cropped['MAG_TOT'])

#create lat2, lon2 arrays of the ridge position. We do this to take advantage of the speed of vector arithmatic in python.
lat2 = np.ones(len(pacific_lat))*ridgelat
lon2 = np.ones(len(pacific_lat))*ridgelon
pacific_dist = haversine_distance(pacific_lon,pacific_lat,lon2,lat2)

plt.figure(1,(15,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(pacific_dist,-1*pacific_bath,color='mediumblue')
ax1.set_title('South Pacific Ridge')
ax1.set_ylabel('Bathymentry, m')
ax1.grid()

ax2 = plt.subplot(2, 1, 2)
ax2.plot(pacific_dist,pacific_mag,color='mediumblue')
ax2.set_xlabel('Distance to Ridge, km')
ax2.set_ylabel('Total magnetic field, nT')

### Bathymetry

Now let's compare the two ridges' bathymetry. 

Let's plot them together on one figure as subplots. Use $\pm 1000$ km as the x-axis limits and -5000 to -1500 meters as the y-axis limits for both ridges. Use code that looks like this `ax1.set_xlim(xxx, xxx)` to set the x and y-axis limits.

<font color=goldenrod>**_Code for you to write_**</font>

In [ ]:
plt.figure(1,(15,10))
ax1 = plt.subplot(2,1,1)
#add code here
ax1.set_xlabel('Distance to Ridge, km') # labels!
ax1.set_ylabel('Bathymetry, m')
ax1.set_title('East Pacific Rise')
ax1.set_xlim(-1000,1000)
#set yaxis limits
ax1.grid()

ax2 = plt.subplot(2,1,2)
#add code here
ax2.set_xlabel('Distance to Ridge, km')
ax2.set_ylabel('Bathymetry, m')
ax2.set_title('Mid Atlantic Ridge')
#set axes limits
ax2.grid()

#plt.tight_layout()

<img src="Figures/spreading_ridges.png" width=900>
> Source: Essentials of Geology (13th Edition) Lutgens, Tarbuck, and Tasa.

**Discussion Questions:** What do you observe in the bathymetry? Do these ridges have a rift valley at the center? Is the slope steep or gentle? Is the bathymetry rough or smooth?

**Write your answer here.**

Based on the ridge bathymetry, which spreading center do you think is spreading faster the Atlantic or Pacific?

**Write your answer here.**

### Write code to remove polynomial fit and plot the East Pacific and Atlantic Magnetic Anomalies

There are long wavelength trends in the magnetic data in addition to the anomalies we seek to understand associated with reversal. These include spatial variability in Earth's magnetic field that induces a field in the rocks on top of the ancient magnetization that is frozen into the rocks.

We can approximate these trends with a linear, or perhaps higher order polynomial, in order to remove them. 

Let's start by fitting a linear model to the Pacific magnetic anomaly data.

In [ ]:
plt.figure(1,(15,10))

ax2 = plt.subplot(2, 1, 2)
ax2.plot(pacific_dist,pacific_mag,color='mediumblue')
ax2.set_xlabel('Distance to Ridge, km')
ax2.set_ylabel('Total magnetic field, nT')
plt.show()

We can use `np.polyfit()` to conduct a linear regression and fit a polynomial to the data. 

In [ ]:
dist_values = pacific_dist
mag_values = pacific_mag
poly_order = 1

np.polyfit(dist_values, mag_values, poly_order)

`np.polyfit()` returns the polynomial coefficients, highest power first. In this case the first value is the slope and the second value is the intercept.

We can assign a variable `poly_coefficients` to be this array.

In [ ]:
poly_coefficients = np.polyfit(dist_values, mag_values, poly_order)

We can then access these coefficients in order with `poly_coefficients[0]` giving us the slope and `poly_coefficients[1]` giving us the intercept.

In [ ]:
poly_coefficients[0]

In [ ]:
poly_coefficients[1]

To remove the higher order polynomial trends is a two step process where first we fit a model to the data, and then we remove the predictions of the model to reveal anomalies. We will use the numpy `np.polyfit()` to determine model coefficients (a, b, etc). 

$Y=a + bX + cX^2 + dX^3 + etc.$

Then we will use numpy `np.poly1d()` to estimate the model for each independent variable to construct an array of model values to remove from the data. Let's write a function to do that.


In [ ]:
def data_detrend(dist_values, mag_values, poly_order):
    """
    Function to fit a polynomial to data, and then remove the 
    fitted polynomial
    
    Parameters
    --------------
    dist_values : distance along profile in km
    mag_values : magnetic data in nT
    poly_order : integer indicating the order of the polynomial fit
        0 (mean), 1 (line), 2 (parabola), 3, etc.
    
    Returns
    ---------------
    model (an array of modeled data), anomaly (an array of corrected data), rms (root mean square error)
    
    usage:
    array1, array2, value=data_detrend(dist_values, mag_values, poly_order)
    """
    
    #np.polyfit() finds the coefficients for a polynomial of specified order
    coefficients = np.polyfit(dist_values,mag_values,poly_order)
    #np.poly1d() constructs a polynomial from the provided coefficients 
    p = np.poly1d(coefficients) 
    #we can calculate the values for that polynomial along the profile
    model = p(dist_values) 
    #we can then subtract those values from the actual values leaving us with the anomaly
    anomaly = mag_values - model
    
    #RMS error
    rms = np.sqrt(np.mean(anomaly**2))
    
    return model, anomaly, rms

## Make a polynomial fit to the Mid-Atlantic data

We want to compare the original data with the polynomial fit. We then want to test different values of the fit to find the best, and then remove the polynomial trend from the data.

First, lets plot the data together with the polynomial fit

<font color=goldenrod>**_Code for you to write_**</font>

In [ ]:
#Apply the function
#plot the model fit
#find the simplest model that provides a good fit.



## Make a polynomial fit to the East Pacific Rise data

Note you may or may not use different polynomial order for the two different data sets. Make sure to give the model and anomaly distinct variable names.

## Plot the anomaly for the Mid-Atlantic Ridge

Now plot the corrected data with the polynomial trend removed. Include a zero reference line: `plot([x1, x2],[y1, y2],color='black')`

Consider distance ranges of -100 to 100 and -200 to 200. Consider putting in xticks every 10 km `ax1.set_xticks(np.arange(-200,200,10))`

<font color=goldenrod>**_Code for you to write_**</font>

In [ ]:
#Plot the anomaly 



## Plot the anomaly for the East Pacific Rise




<img src="Figures/marine_mag_anom.png" width=900>
> Source: Fundamentals of Geophysics (2nd Edition) Lowrie, W.

Zoom in on the plot. 

Which wiggles can you match between lines and to the model profile due to the GPTS above? Can you pick the Bruhnes, Matuyama, Gauss, and Gilbert polarity chrons? 

What distance from the ridge does the Bruhnes-Matuyama reversal (which tells us an age of 776 kyr) occur at for both ridges? 

Zoom in on your plots and write down the distance between the ridge and the Bruhnes-Matuyama reversal for each ridge.

**Write your answer here.**

Define a function that you can use to calculate the spreading rate in km/Myr using the distance from the ridge of the Bruhnes-Matuyama reversal. The function should take the distance to the reversal as input and return a spreading rate. An important piece of information is that the reversal occurred 776,000 years ago (Singer et al. 2019). Make sure that the function has a docstring and that the docstring indicates what units the calculated rate is in.

You can find helpful information about functions here (part of your weekly reading): https://www.inferentialthinking.com/chapters/08/Functions_and_Tables.html

<font color=goldenrod>**_Code for you to write_**</font>

In [ ]:
def spread_rate(dist, age):
    """
    Function to compute the spreading rate
    
    parameters
    ----------
    dist: distance to the ridge in km
    age: age of the reversal in Myr
    
    output
    ------
    rate: spreading rate in km/Myr
    """
    # write your code here
    
    return 

Use your function to compute the spreading rate of the Atlantic and Pacific ridges. Print the results.

Based on the marine magnetic anomalies, which spreading center do you think is spreading faster the Atlantic or Pacific? Is that consistent with your estimate from the bathymetry?

**Write your answer here.**

### Turn in this notebook

Save your completed notebook as html and upload to bcourses.